### Входные данные

У вас имеется поток данных (генератор data_stream). Поля это случайные величины - так сделано для упрощения генерации данных. Есть три поля (названы по уровню сложности задания)

### Задание
##### Мотивация:
У вас есть куча временных рядов, вы хотите научиться предсказывать следующее значение по 1000 предыдущим. 1000 признаков окна это слишком много, однако вы решили заменить их 5ю: средним, дисперсией, минимумом, медианой и максимумом. Однако, все эти признаки надо подсчитать, причём хочется уметь это делать быстро (в течение часа)
##### Для каждого поля нужно сделать следующее:

1. Пробежаться по данным окном размера 1000 (окно сдвигается на 1, то есть следующее окно пересекается с предыдущим по 999 элементам).

2. Для каждого окна посчитайте среднее значение поля и его дисперсию. Делайте yield этих значений, получая генератор tuple. 

3. Для каждого окна найдине минимум, медиану и максимум в нём. Делайте yield этих значений, получая генератор tuple. 

Ответом, который нужно будет засабмитить в гугл форму, является среднее значение tuple по получившемуся потоку, округлённое до **2го знака**.

### Замечания

1. Обратите внимания как генерируются поля. Постарайтесь понять особенность каждого поля и как это можно использовать. Желательно, чтобы для каждого поля у вас было своё решение, максимально эффективно использующее знание об этом поле.
2. Полезные библиотеки: itertools, numpy, collections + всё что найдёте в интернете и можно поставить через pip install
3. **Медианой отсортированного массива arr считайте значение arr[len(arr) // 2]**



Если измерять время работы функций временем работы функции example, то примерное время работы такое:
Одновременно среднее, дисперсия - 1 минута
Одновременно минимум, максимум и медиана:easy - 3 минуты
medium - 6 минут
nightmare - 6 минут


#### Генерация данных

In [2]:
from collections import namedtuple
import random

Record = namedtuple('Record', 'easy medium nightmare')

def data_stream():
    random_generator = random.Random(42)
    easy = 0
    for _ in range(10000000):
        easy += random_generator.randint(0, 2) 
        medium = random_generator.randint(0, 256 - 1)
        nightmare = random_generator.randint(0, 1000000000 - 1)
        
        yield Record(
            easy=easy,
            medium=medium,
            nightmare=nightmare
        )
        
def easy_stream():
    for record in data_stream():
        yield record.easy
        
def medium_stream():
    for record in data_stream():
        yield record.medium
        
def nightmare_stream():
    for record in data_stream():
        yield record.nightmare

#### Подсчёт среднего значения tuple по потоку

In [19]:
from statistics import median

def create_mean_stream(stream):
    out = []
    for i in stream:
        out.append(i)
        if len(out) == 1001:
            out = out[1:]
        if len(out) == 1000:
            yield sum(out) / 1000
            
def create_median_stream(stream):
    out = []
    for i in stream:
        out.append(i)
        if len(out) == 1001:
            out = out[1:]
        if len(out) == 1000:
            yield median(out)

In [27]:
def create_variance_stream(stream):
    out = []
    for i in stream:
        out.append(i)
        if len(out) == 1001:
            out = out[1:]
        if len(out) == 1000:
            yield np.var(np.array(out))

In [14]:
import numpy as np

def get_tuple_stream_mean(stream, number_of_values):
    result = np.zeros(number_of_values, dtype='object')
    count = 0. 
    for streamed_tuple in stream:
        result += streamed_tuple
        count += 1
    return ['{:0.2f}'.format(x) for x in result / count]

In [15]:
%%time
def southen(stream):
    out = []
    for value in stream:
        if len(out) < 1000:
            out.append(i)
            flag = True
        else:
            if flag:
                flag = False
                yield out
            else:
                out = out[1:] + value
                yield out

print(get_tuple_stream_mean(create_mean_stream(easy_stream()), 1))

['4999675.28']
Wall time: 5min 49s


In [16]:
print(get_tuple_stream_mean(create_mean_stream(medium_stream()), 1))
print(get_tuple_stream_mean(create_mean_stream(nightmare_stream()), 1))

['127.48']
['499880345.88']


In [20]:
%%time
print(get_tuple_stream_mean(create_median_stream(medium_stream()), 1))
print(get_tuple_stream_mean(create_median_stream(nightmare_stream()), 1))
print(get_tuple_stream_mean(create_median_stream(easy_stream()), 1))

['127.47']
['499938215.19']
['4999675.28']
Wall time: 47min 33s


In [28]:
%%time
print(get_tuple_stream_mean(create_variance_stream(medium_stream()), 1))
print(get_tuple_stream_mean(create_variance_stream(nightmare_stream()), 1))
print(get_tuple_stream_mean(create_variance_stream(easy_stream()), 1))

['5455.17']
['83228908564027760.00']
['83439.34']
Wall time: 1h 3min 18s


In [23]:
%%time
def example(stream):
    for value in stream:
        yield (value, value + 10)
#print(get_tuple_stream_mean(example(easy_stream()), 2))
#print(get_tuple_stream_mean(example(easy_stream()), 2))
#print(get_tuple_stream_mean(example(easy_stream()), 2))

Wall time: 0 ns


In [17]:
%%time
leng = 0
mean_easy = 0
for i in easy_stream():
    mean_easy = (leng / (leng + 1)) * mean_easy + (1 / (leng + 1)) * i
mean_easy

Wall time: 1min 34s


9999662.0

In [21]:
%%time
min_f = 1e+10
max_f = 1
for i in easy_stream():
    if i < min_f:
        min_f = i
    if i > max_f:
        max_f = i
print("easy - ", min_f, max_f)
min_f = 1e+10
max_f = 1
for i in medium_stream():
    if i < min_f:
        min_f = i
    if i > max_f:
        max_f = i
print("medium - ", min_f, max_f)
min_f = 1e+10
max_f = 1
for i in nightmare_stream():
    if i < min_f:
        min_f = i
    if i > max_f:
        max_f = i
print("nightmare - ", min_f, max_f)

easy -  2 9999662
medium -  0 255
nightmare -  264 999999910
Wall time: 4min 37s
